# Build a Retrieval Augmented Generation (RAG) Model for Zhida Technology

Document from 港交所上市文件-挚达科技-业务【https://www1.hkexnews.hk/app/sehk/2024/106935/2024112801952_c.html】

Reference Links to Learn LangChain:
1. DeepLearning.ai Course: https://learn.deeplearning.ai/courses/langchain-chat-with-your-data
2. LangChain Github: https://github.com/langchain-ai/langchain/blob/master/docs/docs/tutorials

## Retrieval augmented generation
 
In retrieval augmented generation (RAG), an LLM retrieves contextual documents from an external dataset as part of its execution. 

This is useful if we want to ask question about specific documents (e.g., our PDFs, a set of videos, etc).

### Document Loading

Reference: DeepLearning.ai -> https://learn.deeplearning.ai/courses/langchain-chat-with-your-data/lesson/2/document-loading

Document Reference: 01_document_loading.ipynb

In [3]:
import os
import getpass
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
openai.api_key  = os.environ['OPENAI_API_KEY']

Enter your OpenAI API key:  ········


Load PDF (*PyPDFLoader* by *langchain.document_loaders* does NOT support PDFs from 港交所)

Way 1: pdfplumber


In [4]:
#%pip install pypdf

In [5]:
from langchain.document_loaders import PyPDFLoader
import pdfplumber

loader = pdfplumber.open("/Users/lavendashan/Documents/AIML/LangChain-RAG/Files/挚达-业务.pdf")
pages = loader.pages

# ！读取为乱码 ！#
#loader = PyPDFLoader("/Users/lavendashan/Documents/AIML/LangChain-RAG/Files/挚达-业务.pdf")
#pages = loader.load()

### TEST ###
#import pdfplumber
#with pdfplumber.open("/Users/lavendashan/Documents/AIML/LangChain-RAG/Files/挚达-业务.pdf") as pdf:
 #   for page in pdf.pages:
  #      text = page.extract_text()
   #     print(text)

In [6]:
len(pages)

110

In [7]:
### 第一页 ###
page = pages[0]

print(page.extract_text())

本文件為草擬本，屬不完整並可予更改，且本文件須與本文件封面「警告」一節一併閱讀。
業 務
我們的使命
以「充電」為入口，推動電動汽車和綠色能源進入全球的家庭，促進人類可持續
發展。
我們的願景
致力於成為全球電動汽車家庭智能充電和綠色能源科技領軍企業。
概覽
根據弗若斯特沙利文的資料，我們是全球最大的電動汽車家庭充電解決方案供應
商之一。根據弗若斯特沙利文的資料，按於往績記錄期間家用電動汽車充電樁的銷量
計，我們高居全球首位，而按於往績記錄期間家用電動汽車充電樁的銷售額計，我們
位列全球第四。此外，根據弗若斯特沙利文的資料，按於往績記錄期間家用電動汽車
充電樁的銷量及銷售額計，我們在中國排名第一。於2023年，全球及中國的家用電動
汽車充電樁市場的銷售總額分別為人民幣61億元及人民幣28億元。按於往績記錄期間
家用電動汽車充電樁的銷售額計，中國企業佔據主導市場份額，乃提供電動汽車家庭
充電解決方案的領軍企業。有關全球及中國五大電動汽車家庭充電解決方案供應商所
佔市場份額的詳情，請參閱「行業概覽－電動汽車家庭充電解決方案－全球電動汽車
家庭充電解決方案的競爭格局」。
我們以向汽車製造商及用戶提供智慧家用電動汽車充電樁為切入點，開發了由產
品、服務及數字化平台構成的「三位一體」電動汽車家庭充電解決方案。
• 產品：於往績記錄期間，我們全球已累計出貨120萬台家用電動汽車充電樁
及在中國已累計出貨110萬台家用電動汽車充電樁。根據弗若斯特沙利文的
資料，於往績記錄期間，按家用電動汽車充電樁銷量計，我們的中國市場
佔有率達15.6%，全球市場佔有率達到9.5%。我們亦已開發毛利率較高且
收入潛力優厚的先進產品，包括電動汽車充電機器人及EMS解決方案。
• 服務：為了讓我們的智慧家用電動汽車充電樁和數字能源管理服務賦能千
家萬戶，根據弗若斯特沙利文的資料，我們建立了中國最大的充電樁服務
網絡，提供上門安裝及售後服務，截至2024年9月30日，已覆蓋全國超過
360個城市。於往績記錄期間，我們完成合計110萬次安裝及售後服務工
作。我們的數字家庭能源管理服務亦包括共享充電、家庭充電用戶增值服
務等。我們亦開發各種其他服務，為用戶帶來更智能化的生活體驗。
– 160 –


Way 2: LangChain PDFPlumberLoader

Reference: https://python.langchain.com/docs/integrations/document_loaders/pdfplumber/

In [8]:
from langchain_community.document_loaders import PDFPlumberLoader

loader = PDFPlumberLoader("/Users/lavendashan/Documents/AIML/LangChain-RAG/Files/挚达-业务.pdf")
docs = loader.load()

In [9]:
print(docs[0])

page_content='本文件為草擬本，屬不完整並可予更改，且本文件須與本文件封面「警告」一節一併閱讀。
業 務
我們的使命
以「充電」為入口，推動電動汽車和綠色能源進入全球的家庭，促進人類可持續
發展。
我們的願景
致力於成為全球電動汽車家庭智能充電和綠色能源科技領軍企業。
概覽
根據弗若斯特沙利文的資料，我們是全球最大的電動汽車家庭充電解決方案供應
商之一。根據弗若斯特沙利文的資料，按於往績記錄期間家用電動汽車充電樁的銷量
計，我們高居全球首位，而按於往績記錄期間家用電動汽車充電樁的銷售額計，我們
位列全球第四。此外，根據弗若斯特沙利文的資料，按於往績記錄期間家用電動汽車
充電樁的銷量及銷售額計，我們在中國排名第一。於2023年，全球及中國的家用電動
汽車充電樁市場的銷售總額分別為人民幣61億元及人民幣28億元。按於往績記錄期間
家用電動汽車充電樁的銷售額計，中國企業佔據主導市場份額，乃提供電動汽車家庭
充電解決方案的領軍企業。有關全球及中國五大電動汽車家庭充電解決方案供應商所
佔市場份額的詳情，請參閱「行業概覽－電動汽車家庭充電解決方案－全球電動汽車
家庭充電解決方案的競爭格局」。
我們以向汽車製造商及用戶提供智慧家用電動汽車充電樁為切入點，開發了由產
品、服務及數字化平台構成的「三位一體」電動汽車家庭充電解決方案。
• 產品：於往績記錄期間，我們全球已累計出貨120萬台家用電動汽車充電樁
及在中國已累計出貨110萬台家用電動汽車充電樁。根據弗若斯特沙利文的
資料，於往績記錄期間，按家用電動汽車充電樁銷量計，我們的中國市場
佔有率達15.6%，全球市場佔有率達到9.5%。我們亦已開發毛利率較高且
收入潛力優厚的先進產品，包括電動汽車充電機器人及EMS解決方案。
• 服務：為了讓我們的智慧家用電動汽車充電樁和數字能源管理服務賦能千
家萬戶，根據弗若斯特沙利文的資料，我們建立了中國最大的充電樁服務
網絡，提供上門安裝及售後服務，截至2024年9月30日，已覆蓋全國超過
360個城市。於往績記錄期間，我們完成合計110萬次安裝及售後服務工
作。我們的數字家庭能源管理服務亦包括共享充電、家庭充電用戶增值服
務等。我們亦開發各種其他服務，為用戶帶來更智能化的生活體驗。
– 160 –
' metadata={'source': '/Users/lavendashan

In [10]:
### 图片分析不了，只能读取里面的文本 ###
print(docs[3])

page_content='本文件為草擬本，屬不完整並可予更改，且本文件須與本文件封面「警告」一節一併閱讀。
業 務
數字家庭能源管理解決方案
家庭充電控制 綜合家庭能源解決方案
社區共享充電服務
多元化服務 推動電力交易
電動汽車充電樁安裝 碳交易
及售後服務
安裝、配置及管理系統
數字化平台
社區充電共享平台
「車到戶」及「車能
互動」平台
智慧家用
解E 決M 方S 案 充電 電動 機汽 器車 人 充電 電動 樁汽 及車 配件 智能充電硬件設備 我們目前提供的產品及服務
開發中的產品及服務
產品組合
附註： 我們目前提供並不斷升級的產品及服務包括智慧家用電動汽車充電樁及配件、電動汽車充電機器
人及EMS解決方案、社區充電共享平台、安裝、配置及管理系統、電動汽車充電樁安裝及售後服
務、社區共享充電服務以及家庭充電控制。正在開發的產品及服務包括智能充電硬件產品（預期
將於2025年底推出）、「車到戶」及「車能互動」平台（預期將於2025年底推出）、綜合家庭能源解決
方案（預期將於2025年底推出）、碳交易匯集（預期將於2026年推出）以及促進電力交易（預期將於
2026年推出）。
我們的產品組合
我們目前的產品組合主要包括各類智慧家用電動汽車充電樁及相關配件。對汽車
製造商而言，我們基於其對外形、功能及規格等具體要求（包括在電動汽車充電樁外殼
印上相關汽車製造商標誌的要求）提供定制化的智慧家用電動汽車充電樁。對零售客戶
而言，我們在滿足基本家用充電需求的基礎上，還融入了吸引人的設計和功能，使我
們的智慧家用電動汽車充電樁更加貼近消費者的個性化需求。我們為零售客戶提供的
智慧家用電動汽車充電樁印有我們的標誌，並與我們銷售給汽車製造商的具有大致相
同的標準及規格，通過10項技術考核（防水、防塵、防雷電、溫控保護、急停保護、
漏電保護、短路保護、過功保護、過流保護、過╱欠壓保護）。只要用戶選擇合適動
能輸入配置，我們的智慧家用電動汽車充電樁能與主要汽車品牌及型號兼容。於2024
年，我們已推出毛利率較高且收入潛力優厚的先進產品，例如電動汽車充電機器人及
EMS解決方案。為進一步優化我們的產品組合，我們正在大力研發智能充電硬件設備
等創新產品，而我們相信這將成為用戶獲取我們不斷發展的數字家庭能源管理服務的
關鍵途徑。
– 163 –
' metadata={'source

In [11]:
### 再测试一个App相关的文+图 ###
print(docs[27])

page_content='本文件為草擬本，屬不完整並可予更改，且本文件須與本文件封面「警告」一節一併閱讀。
業 務
數字家庭能源管理服務
我們提供日益增長的服務矩陣，旨在對電動汽車家庭充電及家庭能源管理進行數
字化改造。該數字服務允許用戶遠程監控、查看及控制電動汽車家庭充電活動，並通
過公共電動汽車充電樁定位及充電。目前，我們主要通過摯達App及支付寶小程序提
供數字家庭能源管理解決方案。以下截圖展示了摯達App及支付寶小程序的界面及主
要功能。
掃描以綁定
開始充電
電動汽車充電樁 充電詳情
電動汽車
充電樁信息
充電信息
充電設置
充電設置
數據中心 數據中心
累計數據統計
今日累計總
充電量統計
當前累計數據統計
親友累計數據統計
充電電量
摯達App
– 187 –
' metadata={'source': '/Users/lavendashan/Documents/AIML/LangChain-RAG/Files/挚达-业务.pdf', 'file_path': '/Users/lavendashan/Documents/AIML/LangChain-RAG/Files/挚达-业务.pdf', 'page': 27, 'total_pages': 110, 'CreationDate': "D:20241128205451+08'00'", 'Creator': 'Adobe InDesign 19.1 (Windows)', 'ModDate': "D:20241128205512+08'00'", 'Producer': 'Adobe PDF Library 17.0', 'Trapped': 'False'}


In [12]:
### 检查表格读取，应该可以 ###
print(docs[19])

page_content='本文件為草擬本，屬不完整並可予更改，且本文件須與本文件封面「警告」一節一併閱讀。
業 務
下表載列我們於所示期間來自銷售產品及提供服務的收入明細。
截至12月31日止年度 截至9月30日止九個月
2021年 2022年 2023年 2023年 2024年
人民幣元 % 人民幣元 % 人民幣元 % 人民幣元 % 人民幣元 %
（以千計，百分比除外）
（未經審核）
收入
銷售產品......................... 190,130 53.2 407,434 58.5 320,943 47.8 242,397 49.9 238,000 54.1
提供服務......................... 167,477 46.8 289,626 41.5 349,790 52.2 243,424 50.1 202,085 45.9
總計 ............................ 357,607 100.0 697,060 100.0 670,733 100.0 485,821 100.0 440,085 100.0
我們的產品組合
多年來，我們已開發出各種型號的智慧家用電動汽車充電樁及配件，以滿足客戶
對電動汽車家庭充電的需求。我們生產的智慧家用電動汽車充電樁既可由汽車製造商
將其作為其汽車的一部分銷售予客戶，亦可以通過我們的自有品牌於我們的自營零售
渠道及第三方分銷商銷售予用戶。基於我們對汽車製造商技術規格的深入了解，我們
能夠根據其具體要求定制我們的智慧家用電動汽車充電樁。於往績記錄期間，我們的
產品組合所產生的絕大部分收入均來自智慧家用電動汽車充電樁。我們亦已開發毛利
率較高且收入潛力優厚的先進產品，包括電動汽車充電機器人及EMS解決方案。
智慧家用電動汽車充電樁
我們的智慧家用電動汽車充電樁專為方便、高效和安全的家庭充電而設計。為滿
足家庭充電需求，這些智慧家用電動汽車充電樁小巧便攜、易於安裝和使用，並與所
有主要型號的電動汽車兼容。它們提供一系列充電模式，所有模式均可通過移動應用
程序進行控制，以便全天候實時監控和管理。
安全是我們產品設計流程的驅動力。因此，我們為智慧家用電動汽車充電樁配備
了充電保護裝置和功能，使用戶能夠監控、查看和分析家庭用電環境，對任何異常情
況及時發出預警，並提高整體使用效率。


In [13]:
# 提取所有文档的文本内容
all_text = " ".join([doc.page_content for doc in docs])
print(all_text[:500])  # 打印前500个字符作为示例

本文件為草擬本，屬不完整並可予更改，且本文件須與本文件封面「警告」一節一併閱讀。
業 務
我們的使命
以「充電」為入口，推動電動汽車和綠色能源進入全球的家庭，促進人類可持續
發展。
我們的願景
致力於成為全球電動汽車家庭智能充電和綠色能源科技領軍企業。
概覽
根據弗若斯特沙利文的資料，我們是全球最大的電動汽車家庭充電解決方案供應
商之一。根據弗若斯特沙利文的資料，按於往績記錄期間家用電動汽車充電樁的銷量
計，我們高居全球首位，而按於往績記錄期間家用電動汽車充電樁的銷售額計，我們
位列全球第四。此外，根據弗若斯特沙利文的資料，按於往績記錄期間家用電動汽車
充電樁的銷量及銷售額計，我們在中國排名第一。於2023年，全球及中國的家用電動
汽車充電樁市場的銷售總額分別為人民幣61億元及人民幣28億元。按於往績記錄期間
家用電動汽車充電樁的銷售額計，中國企業佔據主導市場份額，乃提供電動汽車家庭
充電解決方案的領軍企業。有關全球及中國五大電動汽車家庭充電解決方案供應商所
佔市場份額的詳情，請參閱「行業概覽－電動汽車家庭充電解決方案－全球電動汽車
家庭充電解決方案的競爭格局」。
我們以向汽車製造商及


In [28]:
# 去掉空格，保留换行符
all_text = all_text.replace(" ", "")

#print(all_text)

### Embedding and VectorStore

Use Way 2: *LangChain PDFPlumberLoader* to continue

通常，**Embedding** 生成向量，**VectorStore** 存储向量并高效检索相关内容。

**Embedding** 和**Vectorize** 区别：<br>
**Embedding** 更注重语义，是一种深层的、上下文感知的向量化方法，适合复杂任务（如语义搜索、推荐系统）。<br>
**Vectorize** 是更通用的向量化技术，可以是简单统计、特征提取，或者深度学习生成的向量。

又是报错 -- LangChain官方的text_splitter不能分割中文😅，那就试试jieba！

**RecursiveCharacterTextSplitter** vs. **Jieba** 区别 <br>
**RecursiveCharacterTextSplitter**:
- 按段落、句子或固定长度的块进行分割。
- 保持上下文连续性，通常分割结果是较大的文本块（句子块或字符块），主要用于生成语言模型的输入片段。

**Jieba**:
- 按单词粒度进行分词，将句子切割成最小的词汇单位。
- 主要用于语义分析、信息检索等需要精准语义单位的场景。

In [29]:
import jieba

seg_list = jieba.cut(all_text, cut_all=False)
seg_list = "/ ".join(seg_list)
print("Default Mode: ", seg_list[:800])  # 精确模式

Default Mode:  本/ 文件/ 為/ 草擬/ 本/ ，/ 屬不/ 完整/ 並/ 可予/ 更改/ ，/ 且/ 本/ 文件/ 須與本/ 文件/ 封面/ 「/ 警告/ 」/ 一節/ 一/ 併/ 閱讀/ 。/ 
/ 業務/ 
/ 我們/ 的/ 使命/ 
/ 以/ 「/ 充電/ 」/ 為/ 入口/ ，/ 推動/ 電動/ 汽車/ 和/ 綠色/ 能源/ 進入/ 全球/ 的/ 家庭/ ，/ 促進/ 人類/ 可持續/ 
/ 發展/ 。/ 
/ 我們/ 的/ 願景/ 
/ 致力/ 於/ 成為/ 全球/ 電動/ 汽車/ 家庭/ 智能/ 充電/ 和/ 綠色/ 能源/ 科技/ 領軍企業/ 。/ 
/ 概覽/ 
/ 根據/ 弗若/ 斯特/ 沙利文/ 的/ 資料/ ，/ 我們/ 是/ 全球/ 最大/ 的/ 電動/ 汽車/ 家庭/ 充電解決/ 方案/ 供應/ 
/ 商/ 之一/ 。/ 根據/ 弗若/ 斯特/ 沙利文/ 的/ 資料/ ，/ 按/ 於/ 往績/ 記錄/ 期間/ 家用/ 電動/ 汽車/ 充電樁/ 的/ 銷量/ 
/ 計/ ，/ 我們/ 高居/ 全球/ 首位/ ，/ 而/ 按/ 於/ 往績/ 記錄/ 期間/ 家用/ 電動/ 汽車/ 充電樁/ 的/ 銷售/ 額計/ ，/ 我們/ 
/ 位列/ 全球/ 第四/ 。/ 此外/ ，/ 根據/ 弗若/ 斯特/ 沙利文/ 的/ 資料/ ，/ 按/ 於/ 往績/ 記錄/ 期間/ 家用/ 電動/ 汽車/ 
/ 充電樁/ 的/ 銷量/ 及/ 銷售/ 額計/ ，/ 我們/ 在/ 中國/ 排名/ 第一/ 。/ 於/ 2023/ 年/ ，/ 全球/ 及/ 中國/ 的/ 家用/ 電動/ 
/ 汽車/ 充電樁/ 市場/ 的/ 銷售/ 總額/ 分別/ 為/ 人民/ 幣/ 61/ 億元及/ 人民/ 幣/ 28/ 億元/ 。/ 按/ 於/ 往績/ 記錄/ 期間/ 
/ 家用/ 電動/ 汽車/ 充電樁


Default Mode: 本/ 文件/ 為/ 草擬/ 本/ ，/ 屬不/ 完整/ 並/ 可予/ 更改/ ，/ 且/ 本/ 文件/ 須與本/ 文件/ 封面/ 「/ 警告/ 」/ 一節/ 一/ 併/ 閱讀/ 。/ 
/ 業務/ 
/ 我們/ 的/ 使命/ 
/ 以/ 「/ 充電/ 」/ 為/ 入口/ ，/ 推動/ 電動/ 汽車/ 和/ 綠色/ 能源/ 進入/ 全球/ 的/ 家庭/ ，/ 促進/ 人類/ 可持續/ 
/ 發展/ 。/ 
/ 我們/ 的/ 願景/ 
/ 致力/ 於/ 成為/ 全球/ 電動/ 汽車/ 家庭/ 智能/ 充電/ 和/ 綠色/ 能源/ 科技/ 領軍企業/ 。/ 
/ 概覽/ 
/ 根據/ 弗若/ 斯特/ 沙利文/ 的/ 資料/ ，/ 我們/ 是/ 全球/ 最大/ 的/ 電動/ 汽車/ 家庭/ 充電解決/ 方案/ 供應/ 
/ 商/ 之一/ 。/ 根據/ 弗若/ 斯特/ 沙利文/ 的/ 資料/ ，/ 按/ 於/ 往績/ 記錄/ 期間/ 家用/ 電動/ 汽車/ 充電樁/ 的/ 銷量/ 
/ 計/ ，/ 我們/ 高居/ 全球/ 首位/ ，/ 而/ 按/ 於/ 往績/ 記錄/ 期間/ 家用/ 電動/ 汽車/ 充電樁/ 的/ 銷售/ 額計/ ，/ 我們/ 
/ 位列/ 全球/ 第四/ 。/ 此外/ ，/ 根據/ 弗若/ 斯特/ 沙利文/ 的/ 資料/ ，/ 按/ 於/ 往績/ 記錄/ 期間/ 家用/ 電動/ 汽車/ 
/ 充電樁/ 的/ 銷量/ 及/ 銷售/ 額計/ ，/ 我們/ 在/ 中國/ 排名/ 第一/ 。/ 於/ 2023/ 年/ ，/ 全球/ 及/ 中國/ 的/ 家用/ 電動/ 
/ 汽車/ 充電樁/ 市場/ 的/ 銷售/ 總額/ 分別/ 為/ 人民/ 幣/ 61/ 億元及/ 人民/ 幣/ 28/ 億元/ 。/ 按/ 於/ 往績/ 記錄/ 期間/ 
/ 家用/ 電動/ 汽車/ 充電樁/ 的/ 銷售/ 額計/ ，/ 中國/ 企業/ 佔據/ 主導市/ 場份額/ ，/ 乃/ 提供/ 電動/ 汽車/ 家庭/ 
/ 充電解決/ 方案/ 的/ 領軍企業/ 。/ 有關/ 全球/ 及/ 中國/ 五大/ 電動/ 汽車/ 家庭/ 充電解決/ 方案/ 供應/ 商所/ 
/ 佔市/ 場份額/ 的/ 詳情/ ，/ 請/ 參閱/ 「/ 行業/ 概覽/ －/ 電動/ 汽車/ 家

感觉分词结果太细了，试试类LangChain RecursiveCharacterTextSplitter的按固定长度分块看看模型上下文理解的效果👀？<br>
（chunk_size 和 chunk_overlap 与 LangChain 参考文档相同）

**可能问题**：
分词结果太细会导致向量数据库存储大量碎片，影响检索效果和生成的上下文质量。<br>
**解决方案**：
将分词后的结果重新组织为更大的块（chunk），例如以一定的字数或句子数为单位进行合并。



In [31]:
chunk_size = 1500  # 每块最多包含 1500 个字符
chunk_overlap = 150

# 实现分块并加入重叠
chunks = [
    all_text[i:i + chunk_size]
    for i in range(0, len(all_text), chunk_size - chunk_overlap)
    if i < len(all_text)  # 确保块起始索引在文本范围内
]
print("按固定长度分块（含重叠）:", chunks[0])

# 对每个块进行分词
segmented_chunks = [" ".join(jieba.cut(chunk)) for chunk in chunks]
print("分词后（含重叠）:", segmented_chunks[0])

按固定长度分块（含重叠）: 本文件為草擬本，屬不完整並可予更改，且本文件須與本文件封面「警告」一節一併閱讀。
業務
我們的使命
以「充電」為入口，推動電動汽車和綠色能源進入全球的家庭，促進人類可持續
發展。
我們的願景
致力於成為全球電動汽車家庭智能充電和綠色能源科技領軍企業。
概覽
根據弗若斯特沙利文的資料，我們是全球最大的電動汽車家庭充電解決方案供應
商之一。根據弗若斯特沙利文的資料，按於往績記錄期間家用電動汽車充電樁的銷量
計，我們高居全球首位，而按於往績記錄期間家用電動汽車充電樁的銷售額計，我們
位列全球第四。此外，根據弗若斯特沙利文的資料，按於往績記錄期間家用電動汽車
充電樁的銷量及銷售額計，我們在中國排名第一。於2023年，全球及中國的家用電動
汽車充電樁市場的銷售總額分別為人民幣61億元及人民幣28億元。按於往績記錄期間
家用電動汽車充電樁的銷售額計，中國企業佔據主導市場份額，乃提供電動汽車家庭
充電解決方案的領軍企業。有關全球及中國五大電動汽車家庭充電解決方案供應商所
佔市場份額的詳情，請參閱「行業概覽－電動汽車家庭充電解決方案－全球電動汽車
家庭充電解決方案的競爭格局」。
我們以向汽車製造商及用戶提供智慧家用電動汽車充電樁為切入點，開發了由產
品、服務及數字化平台構成的「三位一體」電動汽車家庭充電解決方案。
•產品：於往績記錄期間，我們全球已累計出貨120萬台家用電動汽車充電樁
及在中國已累計出貨110萬台家用電動汽車充電樁。根據弗若斯特沙利文的
資料，於往績記錄期間，按家用電動汽車充電樁銷量計，我們的中國市場
佔有率達15.6%，全球市場佔有率達到9.5%。我們亦已開發毛利率較高且
收入潛力優厚的先進產品，包括電動汽車充電機器人及EMS解決方案。
•服務：為了讓我們的智慧家用電動汽車充電樁和數字能源管理服務賦能千
家萬戶，根據弗若斯特沙利文的資料，我們建立了中國最大的充電樁服務
網絡，提供上門安裝及售後服務，截至2024年9月30日，已覆蓋全國超過
360個城市。於往績記錄期間，我們完成合計110萬次安裝及售後服務工
作。我們的數字家庭能源管理服務亦包括共享充電、家庭充電用戶增值服
務等。我們亦開發各種其他服務，為用戶帶來更智能化的生活體驗。
–160–
本文件為草擬本，屬不完整並可予更改，且本文件須與本文件封面「警告」一節一併閱讀。
業務
•數

In [32]:
len(segmented_chunks[0])   # 检查 chunk_size

2370

将 segmented_chunks 转为 LangChain 文档对象，继续向量存储和检索的步骤

In [33]:
from langchain.schema import Document

# 将分词后的文本块转换为 LangChain 文档对象
splits = [Document(page_content=chunk) for chunk in segmented_chunks]
print("LangChain 文档对象:", splits[:2])

LangChain 文档对象: [Document(metadata={}, page_content='本 文件 為 草擬 本 ， 屬不 完整 並 可予 更改 ， 且 本 文件 須與本 文件 封面 「 警告 」 一節 一 併 閱讀 。 \n 業務 \n 我們 的 使命 \n 以 「 充電 」 為 入口 ， 推動 電動 汽車 和 綠色 能源 進入 全球 的 家庭 ， 促進 人類 可持續 \n 發展 。 \n 我們 的 願景 \n 致力 於 成為 全球 電動 汽車 家庭 智能 充電 和 綠色 能源 科技 領軍企業 。 \n 概覽 \n 根據 弗若 斯特 沙利文 的 資料 ， 我們 是 全球 最大 的 電動 汽車 家庭 充電解決 方案 供應 \n 商 之一 。 根據 弗若 斯特 沙利文 的 資料 ， 按 於 往績 記錄 期間 家用 電動 汽車 充電樁 的 銷量 \n 計 ， 我們 高居 全球 首位 ， 而 按 於 往績 記錄 期間 家用 電動 汽車 充電樁 的 銷售 額計 ， 我們 \n 位列 全球 第四 。 此外 ， 根據 弗若 斯特 沙利文 的 資料 ， 按 於 往績 記錄 期間 家用 電動 汽車 \n 充電樁 的 銷量 及 銷售 額計 ， 我們 在 中國 排名 第一 。 於 2023 年 ， 全球 及 中國 的 家用 電動 \n 汽車 充電樁 市場 的 銷售 總額 分別 為 人民 幣 61 億元及 人民 幣 28 億元 。 按 於 往績 記錄 期間 \n 家用 電動 汽車 充電樁 的 銷售 額計 ， 中國 企業 佔據 主導市 場份額 ， 乃 提供 電動 汽車 家庭 \n 充電解決 方案 的 領軍企業 。 有關 全球 及 中國 五大 電動 汽車 家庭 充電解決 方案 供應 商所 \n 佔市 場份額 的 詳情 ， 請 參閱 「 行業 概覽 － 電動 汽車 家庭 充電解決 方案 － 全球 電動 汽車 \n 家庭 充電解決 方案 的 競爭 格局 」 。 \n 我們 以 向 汽 車 製 造 商 及 用 戶 提供 智慧 家用 電動 汽車 充電樁 為 切入 點 ， 開發 了 由產 \n 品 、 服務及 數字化 平台 構成 的 「 三位 一體 」 電動 汽車 家庭 充電解決 方案 。 \n • 產品 ： 於 往績 記錄 期間 ， 我們 全球 已累計 出貨 120 萬台 家用 電動

In [21]:
print("LangChain 文档对象:", splits[-1])

LangChain 文档对象: page_content='.   .   .   .   .   .   2023 年   上   海市 經濟 和 信息化 委員會 、 上海市 
 財 政局 、 國家 稅務 總局 上海市 稅務 
 局 、 中華 人民 共和 國 上海 海關 
 國家 高新 技術 企業   .   .   .   .   .   .   .   .   .   2022 年 、   上   海市 科學 技術 委員會 、 上海市 財政 
 2019 年 及   局 、 國家 稅務 總局 上海市 稅務局 
 2016 年 
 上海市 高新 技術 成果   2022 年   上海市 科學 技術 委員會 
 轉化 認定 項目   .   .   .   .   .   .   .   .   .   .   . 
 上海市 高價值 專利   2022 年   上海市 知識產 權局 
 運營 大賽 20 強 .   .   .   .   .   .   .   .   .   .   . 
 2022 第十一 屆 金磚 價值 榜   2022 年   金磚 論壇 
 年度 科創 先鋒獎   .   .   .   .   .   .   .   .   . 
 2022 中國 充換 電行業   2022 年   中   國充 電樁網 、 中國國際 電動 汽車充 
 十大 最具 投資 價值 品牌 .   .   .   .   換電產業 大會 
 上海市 專精特 新 中小 企業 .   .   .   .   2021 年 、   上海市 經濟 和 信息化 委員會 
 2018 年 及 
 2015 年 
 工信部 專精特 新 小 巨人 企業 .   .   2021 年   中國 工信部 
 –   268   – 
   本 文件 為 草擬 本 ， 屬不 完整 並 可予 更改 ， 且 本 文件 須與本 文件 封面 「 警告 」 一節 一 併 閱讀 。 
 業   務 
 獎項 ╱ 項目   獎項 ╱ 授出 年度   獎項 ╱ 授出 機構 
 中國節 能 協會 碳 交易 產業   2021 年   中國節 能 協會 碳 交易 產業 聯盟 
 聯盟 成員 .   .   .   .   .   .   .   .   .   .   .   .   .   . 

In [34]:
print(f"创建了 {len(splits)} 个 LangChain 文档。")

创建了 66 个 LangChain 文档。


将这些合并后的文档块存储到向量数据库中，方便后续检索

In [20]:
#!rm -rf /Users/lavendashan/Documents/AIML/LangChain-RAG/docs  # remove old database files if any

In [21]:
#!mkdir -p /Users/lavendashan/Documents/AIML/LangChain-RAG/docs/chroma

In [35]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

persist_directory = "/Users/lavendashan/Documents/AIML/LangChain-RAG/docs/chroma"
embedding = OpenAIEmbeddings()

/var/folders/39/694z3dln17j7vbc80pqxd4540000gn/T/ipykernel_13526/1573291789.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


In [36]:
# 存储到向量数据库
vectordb = Chroma.from_documents(
    documents=splits, 
    embedding=embedding, 
    persist_directory=persist_directory
)

# 持久化存储
vectordb.persist()
print("向量数据库已存储成功！")

向量数据库已存储成功！


/var/folders/39/694z3dln17j7vbc80pqxd4540000gn/T/ipykernel_13526/3496855250.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [37]:
print(vectordb._collection.count())

132


### RAG-enabled ChatBot Interface

In [38]:
# 加载存储的向量数据库
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 测试检索
query = "挚达业务的主要内容是什么？"
results = retriever.get_relevant_documents(query)
for doc in results:
    print(doc.page_content)

了 增強 盈利 能力 並 保持 競爭 優勢 ， 我們 專注 於 開 發利潤 空間 高 的 產品 和 擴展 我 
 們 的 智能 能源 解決 方案 套件 。 該 戰略 利用 新產品 的 推出 和 持續 的 技術 創新來 創造 差異 
 化 的 市場 地位 。 我們 創新產品 開發 方法 的 主要 組成 部分 包括 ： 
 • 推出 先 進產品 。 於 2024 年 ， 我們 推出 收入 潛力 優厚 的 先 進產品 ， 包括 電動 
 汽車 充電 機器 人及 EMS 解決 方案 。 該 等 產品 的 利潤率 較 高 ， 符合 消費者 對 
 智能 綜合 能源 解決 方案 日益 增長 的 需求 。 
 • 智能 自動 技術 。 我們 的 研發 工作 優 先考 慮 推 進產品 智能化 ， 尤其 是 在 自動 
 充電 方面 。 我們 正 投資 支持 自動 駕駛 發展 的 技術 ， 旨在 創造 從 車輛 導航 、 
 停車 到 免提 充電 的 全自動 充電體驗 。 該 等 創新 使 我們 處 於 未來 能源管理 趨 
 勢 的 前沿 ， 用戶 操作 時 也 更 方便 。 
 • 先進 的 能源管理 措施 。 我們 亦 正在 探索 先進 的 能源 管理模式 及技術 ， 例如 
 V2H / V2E 能力 可使 電動 汽車 能夠 將存儲 的 電池 能量 反饋 給電網 。 通過 加強 
 我們 在 這領域 的 技術 儲備 ， 我們 能夠 提供 更靈活 的 能源管理 解決 方案 ， 最 
 大限度 地 利用 可 再生能源 ， 並為 用戶 提供 更 穩定 、 高效 的 電力 管理 服務 。 
 • 智能 家庭 能源 管理系 統擴 張 。 我們 使用 專有 的 數字化 平台 ， 正在 擴展 功能 
 以 創建 全面 的 家庭 能源管理 生態 系統 。 我們 的 平台 現階段 支持 數據 可視 
 化 、 遠程 操作 和 控制 、 智能 充放電 、 實時 報警 和 故障 診斷 等 先進 功能 。 我 
 – 176 – 
 本 文件 為 草擬 本 ， 屬不 完整 並 可予 更改 ， 且 本 文件 須與本 文件 封面 「 警告 」 一節 一 併 閱讀 。 
 業務 
 們 計劃 集成 智慧 家用 電動 汽車 充電樁 及 家庭 儲能 控制器 等 設備 為 為系 統入 
 口 ， 為納入 V2H

/var/folders/39/694z3dln17j7vbc80pqxd4540000gn/T/ipykernel_13526/489462413.py:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


In [39]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# 使用 ChatOpenAI 代替 OpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=1)

# 创建 RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=retriever, 
    chain_type="refine"
)

/var/folders/39/694z3dln17j7vbc80pqxd4540000gn/T/ipykernel_13526/3949312870.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o", temperature=1)


In [40]:
# 测试问答
query = "请问这个文档的主要内容是什么？"
answer = qa_chain.run(query)
print(f"回答: {answer}")

/var/folders/39/694z3dln17j7vbc80pqxd4540000gn/T/ipykernel_13526/3361145349.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(query)


回答: 这个文档的主要内容是介绍一家专注于电动汽车家庭充电解决方案的公司的业务目标、产品和服务，以及其成本优化策略。文档指出，公司通过减少冗余和实施数字化解决方案来削减一般和行政开支，推动无纸化办公以实现长期节约。其目的是改善整体成本结构，提高利润率，实现可持续增长和长期财务稳定。

文中还详述了一体化的电动汽车家庭充电解决方案，包括智慧家用电动汽车充电桩及其相关配件、充电机器人、和EMS解决方案。公司通过数字化平台和服务网络提供安装及售后服务、共享充电和数字能源管理服务。公司通过多种销售渠道销售产品，累计交付120万台充电桩，并完成110万次安装和售后服务。

在营收上，公司从产品销售和服务提供中获取收入，并通过产品组合满足客户对电动汽车家庭充电的需求。这些战略和具体措施旨在继续发展公司的市场地位，并支持其在电动汽车家庭充电领域的领导地位。


In [42]:
import gradio as gr

def chatbot(user_input):
    return qa_chain.run(user_input)

# Gradio Web 聊天机器人
iface = gr.Interface(
    fn=chatbot,
    inputs="text",
    outputs="text",
    title="挚达聊天机器人"
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


### 进一步调试

1. 无法检索到机器人相关产品 -- 【解决方案】GPT3.5更新到4o，合并答案

In [29]:
# 测试检索
query = "挚达科技与机器人相关的业务有哪些？"
results = retriever.get_relevant_documents(query)
for doc in results:
    print(doc.page_content)

了 增強 盈利 能力 並 保持 競爭 優勢 ， 我們 專注 於 開 發利潤 空間 高 的 產品 和 擴展 我 
 們 的 智能 能源 解決 方案 套件 。 該 戰略 利用 新產品 的 推出 和 持續 的 技術 創新來 創造 差異 
 化 的 市場 地位 。 我們 創新產品 開發 方法 的 主要 組成 部分 包括 ： 
 • 推出 先 進產品 。 於 2024 年 ， 我們 推出 收入 潛力 優厚 的 先 進產品 ， 包括 電動 
 汽車 充電 機器 人及 EMS 解決 方案 。 該 等 產品 的 利潤率 較 高 ， 符合 消費者 對 
 智能 綜合 能源 解決 方案 日益 增長 的 需求 。 
 • 智能 自動 技術 。 我們 的 研發 工作 優 先考 慮 推 進產品 智能化 ， 尤其 是 在 自動 
 充電 方面 。 我們 正 投資 支持 自動 駕駛 發展 的 技術 ， 旨在 創造 從 車輛 導航 、 
 停車 到 免提 充電 的 全自動 充電體驗 。 該 等 創新 使 我們 處 於 未來 能源管理 趨 
 勢 的 前沿 ， 用戶 操作 時 也 更 方便 。 
 • 先進 的 能源管理 措施 。 我們 亦 正在 探索 先進 的 能源 管理模式 及技術 ， 例如 
 V2H / V2E 能力 可使 電動 汽車 能夠 將存儲 的 電池 能量 反饋 給電網 。 通過 加強 
 我們 在 這領域 的 技術 儲備 ， 我們 能夠 提供 更靈活 的 能源管理 解決 方案 ， 最 
 大限度 地 利用 可 再生能源 ， 並為 用戶 提供 更 穩定 、 高效 的 電力 管理 服務 。 
 • 智能 家庭 能源 管理系 統擴 張 。 我們 使用 專有 的 數字化 平台 ， 正在 擴展 功能 
 以 創建 全面 的 家庭 能源管理 生態 系統 。 我們 的 平台 現階段 支持 數據 可視 
 化 、 遠程 操作 和 控制 、 智能 充放電 、 實時 報警 和 故障 診斷 等 先進 功能 。 我 
 – 176 – 
 本 文件 為 草擬 本 ， 屬不 完整 並 可予 更改 ， 且 本 文件 須與本 文件 封面 「 警告 」 一節 一 併 閱讀 。 
 業務 
 們 計劃 集成 智慧 家用 電動 汽車 充電樁 及 家庭 儲能 控制器 等 設備 為 為系 統入 
 口 ， 為納入 V2H

In [30]:
# 测试问答
query = "挚达有哪些机器人产品？"
answer = qa_chain.run(query)
print(f"回答: {answer}")

回答: 挚达在推进其机器人产品和相关解决方案方面具有多样化的产品线和战略。以下是其主要产品和策略：

1. **电动汽车充电机器人及EMS解决方案**：挚达计划于2024年推出高利润的电动汽车充电机器人和能源管理系统（EMS）解决方案，以满足消费者对智能综合能源解决方案日益增长的需求。

2. **智能自动技术**：挚达研发的重点包括推进产品智能化，特别是在自动充电方面。他们正在投资支持自动驾驶技术的发展，目标是实现车辆从导航、停车到免提充电的全自动体验。

3. **先进的能源管理措施**：挚达探索包括V2H（车到家）/V2E（车到设备）能力在内的先进能源管理模式，这些技术使电动汽车能够将储存的电池能量反馈给电网。这将最大化可再生能源的利用，并为用户提供稳定高效的电力管理服务。

4. **智能家庭能源管理系统**：通过专有的数字化平台，挚达扩展了家用能源管理的功能，包括数据可视化、远程操作和控制、智能充放电、实时报警及故障诊断等。他们计划集成智慧家用电动汽车充电桩及家庭储能控制器，为V2H/V2E功能奠定基础，并进军绿电及碳交易新兴市场。

挚达通过专注于高利润创新产品和不断技术改进，不仅提升了盈利能力，还巩固了其在快速发展的数字能源管理市场中的竞争地位。


2. 非multimodal -- 无法读图

In [31]:
# 测试检索
query = "挚达的APP包含什么内容？"
results = retriever.get_relevant_documents(query)
for doc in results:
    print(doc.page_content)

本 文件 為 草擬 本 ， 屬不 完整 並 可予 更改 ， 且 本 文件 須與本 文件 封面 「 警告 」 一節 一 併 閱讀 。 
 業務 
 我們 的 使命 
 以 「 充電 」 為 入口 ， 推動 電動 汽車 和 綠色 能源 進入 全球 的 家庭 ， 促進 人類 可持續 
 發展 。 
 我們 的 願景 
 致力 於 成為 全球 電動 汽車 家庭 智能 充電 和 綠色 能源 科技 領軍企業 。 
 概覽 
 根據 弗若 斯特 沙利文 的 資料 ， 我們 是 全球 最大 的 電動 汽車 家庭 充電解決 方案 供應 
 商 之一 。 根據 弗若 斯特 沙利文 的 資料 ， 按 於 往績 記錄 期間 家用 電動 汽車 充電樁 的 銷量 
 計 ， 我們 高居 全球 首位 ， 而 按 於 往績 記錄 期間 家用 電動 汽車 充電樁 的 銷售 額計 ， 我們 
 位列 全球 第四 。 此外 ， 根據 弗若 斯特 沙利文 的 資料 ， 按 於 往績 記錄 期間 家用 電動 汽車 
 充電樁 的 銷量 及 銷售 額計 ， 我們 在 中國 排名 第一 。 於 2023 年 ， 全球 及 中國 的 家用 電動 
 汽車 充電樁 市場 的 銷售 總額 分別 為 人民 幣 61 億元及 人民 幣 28 億元 。 按 於 往績 記錄 期間 
 家用 電動 汽車 充電樁 的 銷售 額計 ， 中國 企業 佔據 主導市 場份額 ， 乃 提供 電動 汽車 家庭 
 充電解決 方案 的 領軍企業 。 有關 全球 及 中國 五大 電動 汽車 家庭 充電解決 方案 供應 商所 
 佔市 場份額 的 詳情 ， 請 參閱 「 行業 概覽 － 電動 汽車 家庭 充電解決 方案 － 全球 電動 汽車 
 家庭 充電解決 方案 的 競爭 格局 」 。 
 我們 以 向 汽 車 製 造 商 及 用 戶 提供 智慧 家用 電動 汽車 充電樁 為 切入 點 ， 開發 了 由產 
 品 、 服務及 數字化 平台 構成 的 「 三位 一體 」 電動 汽車 家庭 充電解決 方案 。 
 • 產品 ： 於 往績 記錄 期間 ， 我們 全球 已累計 出貨 120 萬台 家用 電動 汽車 充電樁 
 及 在 中國 已 累計 出貨 110 萬台 家用 電動 汽車 充電樁 。 根據 弗若 斯特 沙利文 的 
 資料 ， 於 往績

In [32]:
# 测试问答
query = "挚达的APP包含什么内容？"
answer = qa_chain.run(query)
print(f"回答: {answer}")

回答: 挚达的APP主要内容围绕其提供的综合电动汽车家庭充电解决方案，结合产品、服务和数字化平台。具体包括：

1. **产品管理**：用户可以通过APP管理其智慧家用电动汽车充电桩，以及相关配件。这些充电桩支持便捷、高效、安全的充电。

2. **高级产品发布**：自2024年起，挚达引入了新产品，如电动汽车充电机器人和EMS解决方案，以提供更高的毛利率和丰富的收入机会。

3. **服务支持**：APP提供安装和售后服务的信息，帮助用户获取充电桩安装、售后服务及共享充电服务。此外，还支持数字能源管理服务。

4. **数字化平台功能**：支持家庭能源管理的智能化整合，帮助用户将充电需求与家庭能源管理相结合。

5. **销售渠道**：通过APP，用户可以了解公司产品的不同销售渠道，包括面向汽车制造商的直销、零售客户的直销以及通过第三方分销商销售。

通过这些功能，挚达的APP致力于为用户提供便捷全面的家庭充电解决方案和能源管理选择。


In [33]:
# 测试检索
query = "摯達App包含什么内容？"
results = retriever.get_relevant_documents(query)
for doc in results:
    print(doc.page_content)

止 服務協 
 議為止 。 除 收集 上述 用戶 個 人 信息 外 ， 我們 的 數字化 平台 亦 收集 與 電動 汽車 充電樁 生 
 產 、 安裝及 使用 相關 的 業務數 據 ， 用 於 業務 管理 及 分析 。 該 等 業務 信息 被 視為 非個 人 
 性質 ， 目前 為業務 發展 目的 作 永久 保留 。 我們 在 中國境 內 的 日常 經營產生 或 收集 的數 
 據 在位 於 中國 的 服務器 進行 處理 和 存儲 ， 而 在 海外 產生 或 收集 的 數據則 在位 於 海外 的 
 服務器 進行 處理 和 存儲 。 我們 制定 並實施 了 嚴格 的 集團 範圍 內 的 數據 收集 、 處理 和 使 
 用 政策 。 我們 已 制定 信息 系統 數據 安全 管理制度 、 數據 安全 管理 規定 及 信息 分類 和 分 
 類 管理程序 ， 規範 整個 組織 的 數據 安全 。 該 等 文件 概括 在 數據 處理 過程 中針 對 不同 類 
 型 及級 別 的 數據 所 採取 的 安全措施 ， 以確 保資料 安全 。 我們還 制定 了 嚴格 的 內部協 
 議 ， 根據 該 協議 ， 我們 對 數據 進行 分類 訪問 ， 只 允許 有限 的 員 工訪問 最低 限度 的數 
 據 ， 並嚴格 規定 了 分層 訪問 權限 。 我們 的 用戶 訪問 管理程序 規定員 工訪 問權 限及 密碼 
 的 管理 要求 。 此外 ， 我們 已 制定 信息安全 管理 手冊 ， 明確員工 必須 遵守 的 信息安全 
 規範 。 在 個 人 信息 保護 方面 ， 我們 已 實施個 人 信息 保護 管理制度 。 該 制度 界定 組織責 
 – 243 – 
 本 文件 為 草擬 本 ， 屬不 完整 並 可予 更改 ， 且 本 文件 須與本 文件 封面 「 警告 」 一節 一 併 閱讀 。 
 業務 
 任 ， 概括 處理個 人 信息 的 安全措施 ， 並解 決訪 問請求 、 授權 、 影響 評估 及 隱私 保護 等 
 問題 ， 亦 包括 培訓 、 合規 管理 、 突發 事件 應對 及 未成年人 信息 保護 。 為進 一步 增強 安 
 全性 ， 我們 已 實施 計算機 網絡維護 和 管理 措施 及 信息 系統 訪問 和 使用 監控 管理程序 中 
 概述 的 措施 。 為 

In [34]:
# 测试问答
query = "摯達App包含什么内容？"
answer = qa_chain.run(query)
print(f"回答: {answer}")

回答: Based on the additional context provided, the "摯達App" seems to be associated with a company involved in the electric vehicle industry, possibly relating to the production and installation of charging stations. The company is focused on compliance, anti-bribery, and anti-corruption policies, as well as employee agreements regarding confidentiality and non-compete clauses. The context also highlights the importance of insurance, employee safety, seasonality in business operations, and property leasing concerns.

Therefore, the "摯達App" might include functionalities related to managing these aspects, such as employee and operational data security, legal compliance, insurance management, and possibly features related to handling business seasonality and property management. The app could potentially be a tool for overseeing and aligning business practices with regulatory requirements and operational strategies.


3. Hallucination (wrong answer) -- 【解决方案】在 qa_chain 加入 chain_type="refine" 的合并答案方式，解决了乱回答的问题但是回答时间变长 && 不能自动调用 ChatGPT 的 RAG (By default, most of LangChain’s built-in RetrievalQA prompts encourage the LLM to say “I don’t know” if the retrieval doesn’t yield relevant context.)

In [35]:
# 测试问答
query = "挚达科技成立于哪一年？"
results = retriever.get_relevant_documents(query)
for doc in results:
    print(doc.page_content)

tion 
 上海市 企業 技術 中心 ....... 2023 年 上海市 經濟 和 信息化 委員會 、 上海市 
 財 政局 、 國家 稅務 總局 上海市 稅務 
 局 、 中華 人民 共和 國 上海 海關 
 國家 高新 技術 企業 ......... 2022 年 、 上海市 科學 技術 委員會 、 上海市 財政 
 2019 年 及局 、 國家 稅務 總局 上海市 稅務局 
 2016 年 
 上海市 高新 技術 成果 2022 年 上海市 科學 技術 委員會 
 轉化 認定 項目 ........... 
 上海市 高價值 專利 2022 年 上海市 知識產 權局 
 運營 大賽 20 強 ........... 
 2022 第十一 屆 金磚 價值 榜 2022 年金 磚論壇 
 年度 科創 先鋒獎 ......... 
 2022 中國 充換 電行業 2022 年 中國 充電樁網 、 中國國際 電動 汽車充 
 十大 最具 投資 價值 品牌 .... 換電產業 大會 
 上海市 專精特 新 中小 企業 .... 2021 年 、 上海市 經濟 和 信息化 委員會 
 2018 年 及 
 2015 年 
 工信部 專精特 新 小 巨人 企業 .. 2021 年 中國 工信部 
 – 268 – 
 本 文件 為 草擬 本 ， 屬不 完整 並 可予 更改 ， 且 本 文件 須與本 文件 封面 「 警告 」 一節 一 併 閱讀 。 
 業務 
 獎項 ╱ 項目 獎項 ╱ 授出 年度 獎項 ╱ 授出 機構 
 中國節 能 協會 碳 交易 產業 2021 年 中國節 能 協會 碳 交易 產業 聯盟 
 聯盟 成員 ............... 
 上海市 工業 設計 中心 ....... 2020 年 上海市 經濟 和 信息化 委員會 
 2020 中國 年度 數字 技術 2020 年 上海 現代 服務業 聯合會 
 之星 產品獎 ............. 
 執行 主席 成員 ............. 2019 年 同濟 大學 校友 產業 創新 聯盟 
 TopDigital 金獎 與 特別獎 .... 2017 年 TopDigital 
 – 269 – 

tion 
 上海市 企業 技術 中心 ....... 2023 年 上海市 經濟 和 

In [36]:
# 测试问答
query = "挚达科技成立于哪一年？"
answer = qa_chain.run(query)
print(f"回答: {answer}")

回答: 根据提供的信息，挚达科技成立于2014年8月。
